### Add data to Vector Store. I am using LanceDB as Vector Store. They provide $100 free credits.

In [1]:
import os
import lancedb
import json
import pyarrow as pa

In [2]:
chunk_dir = "../../data/chunks"
chunk_files = os.listdir(chunk_dir)

In [ ]:
uri="db://screenplay-agent-msgtqr",
api_key="sk_WZI5Y7IPRBEPBIM7JTOQMWAQDGUM22643EDAWLBZSOJ2Z3SBRMWA====",
region="us-east-1"
table_name = "documents"

In [3]:
db = lancedb.connect(
  uri=uri,
  api_key=api_key,
  region=region
)

In [5]:
schema = pa.schema([
    pa.field("title", pa.string()),
    pa.field("id", pa.string()),
    pa.field("text", pa.string()),
    pa.field("text_vector", pa.list_(pa.float32())),
])
table = db.create_table(table_name, schema=schema)

### Add the data in batches

(Tried using iterator/generator. But ran into issues.)

In [6]:
batch = []
batch_size = 2000
for ind, filename in enumerate(chunk_files):
    chunk_path = os.path.join(chunk_dir, filename)
    with open(chunk_path, 'r', encoding='utf-8') as f:
        chunks = json.load(f)

    for chunk in chunks:
        batch.append(chunk)

    if len(batch) >= batch_size:
        table.add(batch)
        batch = []

# Add remaining records
if batch:
    table.add(batch)

In [14]:
table.schema

title: string
id: string
text: string
vector: fixed_size_list<item: float>[768]
  child 0, item: float

In [13]:
table.alter_columns({"path": "text_vector", "rename": "vector"}) # Renaming column since lancedb looks for column name with vector for performing vector search
table.alter_columns({"path": "vector", "data_type": pa.list_(pa.float32(), 768)})  # Adding dimension since I didn't mention dimension in schema while creating the table

## Search

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
# table = db.open_table(table_name)

In [ ]:
text_query = "12 Monkeys parking lot airport"
vector_query = model.encode(text_query)

In [15]:
table.search(query_type="hybrid").vector(vector_query).text(text_query).to_pandas()

,title,id,text,vector,_relevance_score
0,12 Monkeys,12 Monkeys_184,EXT. PARKING LOT/AIRPORT\nAs YOUNG COLE'S PARE...,"[0.01629723, 0.021248307, 0.02497799, 0.024073...",0.032018
1,12 Monkeys,12 Monkeys_1,\nTwelve Monkeys\nTWELVE MONKEYS\nAn original ...,"[0.06492284, 0.0074991644, 0.025059044, -0.010...",0.016393
2,Absolute Power,Absolute Power_172,INT. AIRPORT LOUNGE -,"[0.016631953, -0.0723681, 0.0043680794, 0.0220...",0.016129
3,10 Things I Hate About You,10 Things I Hate About You_65,EXT. PARKING LOT - DAY Joey leans against Patr...,"[0.020487228, 0.039602064, 0.0115076415, 0.049...",0.016129
4,Elvis,Elvis_323,INTERNATIONAL HOTEL - PARKING GARAGE - DAY\nAn...,"[0.016713805, 0.013903926, -0.001708867, 0.079...",0.015873
5,12 and Holding,12 and Holding_95,EXT. 7-11 PARKING LOT - LATER THAT NIGHT\nMale...,"[-0.0015742738, -0.028643304, 0.01565392, 0.03...",0.015873
6,Deep Cover,Deep Cover_122,EXT. THEATER PARKING LOT/,"[0.0017522366, -0.049317118, 0.0017435338, 0.0...",0.015625
7,Blackberry,Blackberry_66,"EXT. RUNWAY, PEARSON","[-0.0072112437, -0.034491576, -0.00065408176, ...",0.015385
8,JFK,JFK_9,INTERCUT with the Zapruder and Nix films on J....,"[-0.025056802, -0.022652894, -0.0024887817, 0....",0.015385
9,Ricky Stanicky,Ricky Stanicky_9,"EXT. BURBANK AIRPORT - LATER\nTed, Wes and JT ...","[0.024349837, -0.056931645, 0.015563837, 0.037...",0.015152
